# Calculate Partisan Lean for each legislative session

In [7]:
import pandas as pd
import numpy as np

In [8]:
bills =  pd.read_csv("../references/external/bill.csv", sep=";", encoding="latin1", parse_dates=True).rename(columns={'id': 'bill_id'})
bv = pd.read_csv("../references/external/bill_version.csv", sep=";", encoding="latin1", parse_dates=True)
bills.shape

(247565, 17)

In [9]:
#bill= bills.merge(bv['bill_id'], on = "bill_id")
#bills.shape

In [11]:
sessions =  pd.read_csv("../references/external/session.csv", sep=";", encoding="latin1", parse_dates=True)
divs =  pd.read_csv("../references/external/division.csv", sep=";", encoding="latin1", parse_dates=True)
votes =  pd.read_csv("../references/external/vote.csv", sep=";", encoding="latin1", parse_dates=True)
bill_leg_vote =  pd.read_csv("../references/external/bill_legislator_vote.csv", sep=";", encoding="latin1", parse_dates=True)
vote_rec_type =  pd.read_csv("../references/external/vote_recorded_type.csv", sep=";", encoding="latin1", parse_dates=True)


In [12]:
#### Calculate some derived tables:

In [13]:
# Remove missing values
bills = bills[~bills['chamber_id'].isna()]
bills.head()
bills['chamber_id'] = bills['chamber_id'].astype(int)
bills['sc_id'] = bills['session_id'].astype(str) + "-" + bills['chamber_id'].astype(str)

In [14]:
bills.loc[bills['signed'].isna(), 'signed'] = 0
bills.loc[bills['signed'] != 0, 'signed'] = 1
bills['signed'] = bills.signed.astype(int)

bills.loc[bills['passed_lower'].isna(), 'passed_lower'] = 0
bills.loc[bills['passed_lower'] != 0, 'passed_lower'] = 1
bills['passed_lower'] = bills.passed_lower.astype(int)

bills.loc[bills['passed_upper'].isna(), 'passed_upper'] = 0
bills.loc[bills['passed_upper'] != 0, 'passed_upper'] = 1
bills['passed_upper'] = bills.passed_upper.astype(int)
#


In [15]:
np.mean(bills.signed)

0.12801538188104894

In [16]:
passage_percentage = bills[['signed', 'sc_id']].groupby('sc_id').agg(['mean', 'count']).reset_index()
passage_percentage.columns = ['sc_id', 'mean', 'count']
passage_percentage.to_csv('passage_percentage.csv', index=False)

In [17]:
passage_percentage.head()

,sc_id,mean,count
0,436-1,0.000000,26
1,469-2,0.000000,4
2,475-1,0.064026,2171
3,475-2,0.054402,2408
4,520-1,0.106355,1542


In [18]:
sum(passage_percentage['count'])

247564

In [19]:
unique_bills = pd.DataFrame(bv['bill_id'].unique(), columns=['bill_id'])
#sessions = sessions.merge(divs.rename(columns={'id':"division_id"}), on="division_id", how='left')
sessions = sessions.rename(columns={'id':'session_id'})
votes2 = votes.merge(sessions, on = ['session_id', 'division_id'], how='left')
votes2 = votes2[['id', 'division_id', 'session_id', 'bill_id', "chamber_id", 'yes', 'no', 'other']]

In [20]:
votes2.head()

,id,division_id,session_id,bill_id,chamber_id,yes,no,other
0,1,1421.0,1,6,1,21,1,13.0
1,2,1421.0,1,6,2,64,2,38.0
2,3,1421.0,1,7,1,27,0,8.0
3,4,1421.0,1,7,1,25,0,10.0
4,5,1421.0,1,7,1,22,0,13.0


#### Derive liberal/conservate votes from person, party, specific bill vote data


In [22]:
party =  pd.read_csv("../references/external/party.csv", sep=";", encoding="latin1", parse_dates=True)
party = party.rename(columns={"id": "party_id"})
persons =  pd.read_csv("../references/external/person.csv", sep=";", encoding="latin1", parse_dates=True)

In [23]:
# Marking all as liberal except "Republicans", Republicans are a majority of persons
party["is_liberal"] = 1
conservatives = set(["Republican", "Republican \n"])
mask = party["name"].isin(conservatives)
party.loc[mask, "is_liberal"] = 0
party.head()
persons= persons.merge(party[["party_id", "is_liberal"]], left_on = "party_id", right_on = "party_id")
persons = persons.rename(columns = {"id": "person_id"})

In [24]:
vote_rec_type = vote_rec_type.rename(columns={"id": "vote_recorded_type_id"})
blv = bill_leg_vote.merge(vote_rec_type, left_on = 'vote_recorded_type_id', right_on = "vote_recorded_type_id")
blv = blv.merge(persons[["person_id", "is_liberal"]], on = "person_id")

In [25]:
# Table with each vote and whether the person is tagged as liberal or not:
blv.head()

,id,bill_id,vote_id,person_id,vote_recorded_type_id,name,is_liberal
0,1,349728,97964,4018,2,yes,0
1,35,349731,97968,4018,2,yes,0
2,69,349722,97960,4018,2,yes,0
3,103,349876,98058,4018,2,yes,0
4,137,350008,98120,4018,2,yes,0


In [26]:
lib_vote_count = blv[['bill_id', 'vote_id', 'is_liberal']].groupby(['bill_id', 'vote_id']).agg(['sum', 'count'])
lib_vote_count = lib_vote_count.reset_index()
lib_vote_count.columns = ['bill_id', 'vote_id', 'n_lib_votes', 'total_votes']
lib_vote_count.head()

,bill_id,vote_id,n_lib_votes,total_votes
0,4493,209038,15,29
1,4493,209041,5,16
2,4493,209042,5,16
3,4494,272441,5,16
4,4494,272442,5,16


In [27]:
votes3 = votes2.merge(lib_vote_count, on = ['bill_id'], how='left')
votes3 = votes3.merge(unique_bills, on = 'bill_id')
votes3 = votes3.merge(divs[['id', 'abbr']], left_on='division_id', right_on='id')

In [28]:
# Partisan lean is calculated as the ratio of total liberal votes divided by total votes:
total_votes = votes3[['abbr', 'division_id', 'session_id', 'chamber_id', 'yes', 'n_lib_votes', 'total_votes']].groupby(['abbr', 'division_id', 'session_id', 'chamber_id']).agg(['count', 'sum'])
a = total_votes['n_lib_votes']['sum'] / total_votes['total_votes']['sum'] 
a = a.to_frame().reset_index().rename(columns={"sum":"partisan_lean"})
a = a[a.chamber_id != 153]
a = a[a.abbr != 'PR']
a.head()

,abbr,division_id,session_id,chamber_id,partisan_lean
0,AK,651.0,610,1,NaN
1,AK,651.0,610,2,NaN
2,AL,1421.0,502,1,0.339564
3,AL,1421.0,502,2,0.364537
4,AL,1421.0,608,1,0.258371


In [29]:
# Missing vote data from some states. Use the ratio of liberals to chamber size
# From Wikipedia, hard coding the partisan lean from each state:
a.loc[(a['abbr'] == 'AK') & (a['chamber_id'] == 1), 'partisan_lean'] = 15/40
a.loc[(a['abbr'] == 'AK') & (a['chamber_id'] == 2), 'partisan_lean'] = 7/20
a.loc[(a['abbr'] == 'ME') & (a['chamber_id'] == 1), 'partisan_lean'] = 88/151
a.loc[(a['abbr'] == 'ME') & (a['chamber_id'] == 2), 'partisan_lean'] = 21/35
a.loc[(a['abbr'] == 'MO') & (a['chamber_id'] == 1), 'partisan_lean'] = 48/163
a.loc[(a['abbr'] == 'MO') & (a['chamber_id'] == 2), 'partisan_lean'] = 10/34
a.loc[(a['abbr'] == 'MT') & (a['chamber_id'] == 1), 'partisan_lean'] = 41/100
a.loc[(a['abbr'] == 'MT') & (a['chamber_id'] == 2), 'partisan_lean'] = 20/50
a.loc[(a['abbr'] == 'OK') & (a['chamber_id'] == 1), 'partisan_lean'] = 24/101
a.loc[(a['abbr'] == 'OK') & (a['chamber_id'] == 2), 'partisan_lean'] = 9/39
a.loc[(a['abbr'] == 'SC') & (a['chamber_id'] == 1), 'partisan_lean'] = 45/124
a.loc[(a['abbr'] == 'SC') & (a['chamber_id'] == 2), 'partisan_lean'] = 19/46
a.loc[(a['abbr'] == 'UT') & (a['chamber_id'] == 1), 'partisan_lean'] = 16/75
a.loc[(a['abbr'] == 'UT') & (a['chamber_id'] == 2), 'partisan_lean'] = 6/29
a.loc[(a['abbr'] == 'VA') & (a['chamber_id'] == 1), 'partisan_lean'] = 55/100
a.loc[(a['abbr'] == 'VA') & (a['chamber_id'] == 2), 'partisan_lean'] = 21/40

In [30]:
a

,abbr,division_id,session_id,chamber_id,partisan_lean
0,AK,651.0,610,1,0.375000
1,AK,651.0,610,2,0.350000
2,AL,1421.0,502,1,0.339564
3,AL,1421.0,502,2,0.364537
4,AL,1421.0,608,1,0.258371
...,...,...,...,...,...
153,WV,190272.0,637,2,0.424766
154,WY,192572.0,585,1,0.113212
155,WY,192572.0,585,2,0.160087
156,WY,192572.0,639,1,0.121591


In [31]:
derived = "../references/derived/"
import os
if not os.path.exists(derived):
    os.makedirs(derived)
a.to_csv(derived + "partisan_lean.csv")